In [1]:
!pip install pyspark

In [2]:
import pyspark

In [3]:
!pip install kaggle

In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [5]:
!pip install --upgrade kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.17
    Uninstalling kaggle-1.6.17:
      Successfully uninstalled kaggle-1.6.17


In [6]:
!mkdir -p ~/.kaggle

In [7]:
!cp "/content/drive/My Drive/kaggle.json" /root/.kaggle/kaggle.json

In [8]:
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d uciml/autompg-dataset --force

Dataset URL: https://www.kaggle.com/datasets/uciml/autompg-dataset
License(s): CC0-1.0
  0% 0.00/6.31k [00:00<?, ?B/s]
100% 6.31k/6.31k [00:00<00:00, 21.0MB/s]


In [10]:
import os
folder_path = "/content/drive/My Drive/data"
os.makedirs(folder_path, exist_ok=True)

In [11]:
!unzip autompg-dataset.zip -d "/content/drive/My Drive/data"

Archive:  autompg-dataset.zip
replace /content/drive/My Drive/data/auto-mpg.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/drive/My Drive/data/auto-mpg.csv  


In [12]:
import os
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import split, count, when, isnan, col, regexp_replace
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors

for dirname, _, filenames in os.walk('/kaggle/input'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

In [13]:
spark = SparkSession.builder.appName('First Session').getOrCreate()

print(f"Spark Version ({format(spark.version)})")

Spark Version (3.5.5)


In [14]:
file_path = '/content/drive/My Drive/data/auto-mpg.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True,nanValue='?')
df.show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|     1|         ford torino|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70|     1|    ford galaxie 500|
|14.0|        8|       454.0|     220.0|  4354|         9.0|        70|     1|    chevrolet impala|


In [15]:
df.dtypes



[('mpg', 'double'),
 ('cylinders', 'int'),
 ('displacement', 'double'),
 ('horsepower', 'double'),
 ('weight', 'int'),
 ('acceleration', 'double'),
 ('model year', 'int'),
 ('origin', 'int'),
 ('car name', 'string')]

In [16]:
def check_missing(dataframe):
  return dataframe.select([count(when(isnan(col(c)) | col(c).isNull(), c)).alias(c) for c in dataframe.columns]).show()

#check_missing(df)

In [17]:
df.dropna().show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|     1|         ford torino|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70|     1|    ford galaxie 500|
|14.0|        8|       454.0|     220.0|  4354|         9.0|        70|     1|    chevrolet impala|


In [18]:
#col_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'car name']
#for col in col_names:
#  cols=col.split(sep=" ")
#  colums=''
#  for c in cols:
#    colums += c + "_"
#  print(colums)
#df = df.withColumnRenamed(col, colums)
#df.show()
df.show(10)
#df = df.withColumnRenamed('car name', regexp_replace('car name', ' ', '_'))
df.show()
#cols

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|     1|         ford torino|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70|     1|    ford galaxie 500|
|14.0|        8|       454.0|     220.0|  4354|         9.0|        70|     1|    chevrolet impala|


In [19]:
#regexp_replace(col(df), ' ', '_')

In [20]:
df = df.withColumn('horsepower', col('horsepower').cast('int'))
df.dtypes

[('mpg', 'double'),
 ('cylinders', 'int'),
 ('displacement', 'double'),
 ('horsepower', 'int'),
 ('weight', 'int'),
 ('acceleration', 'double'),
 ('model year', 'int'),
 ('origin', 'int'),
 ('car name', 'string')]

In [21]:
numeric_cols= [c for c,t in df.dtypes if t.startswith('string')==False]
df[numeric_cols].summary().show()

+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|summary|               mpg|         cylinders|      displacement|        horsepower|           weight|      acceleration|        model year|            origin|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|  count|               398|               398|               398|               398|              398|               398|               398|               398|
|   mean|23.514572864321615| 5.454773869346734|193.42587939698493|102.89447236180905|2970.424623115578|15.568090452261291| 76.01005025125629|1.5728643216080402|
| stddev| 7.815984312565783|1.7010042445332123|104.26983817119587| 40.26954421185716|846.8417741973268| 2.757688929812676|3.6976266467325862|0.8020548777266148|
|    min|               9.0|      

In [22]:
df.printSchema()

root
 |-- mpg: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- model year: integer (nullable = true)
 |-- origin: integer (nullable = true)
 |-- car name: string (nullable = true)



In [23]:
df.filter(df['acceleration']>15).show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|22.0|        6|       198.0|        95|  2833|        15.5|        70|     1|     plymouth duster|
|18.0|        6|       199.0|        97|  2774|        15.5|        70|     1|          amc hornet|
|21.0|        6|       200.0|        85|  2587|        16.0|        70|     1|       ford maverick|
|26.0|        4|        97.0|        46|  1835|        20.5|        70|     2|volkswagen 1131 d...|
|25.0|        4|       110.0|        87|  2672|        17.5|        70|     2|         peugeot 504|
|25.0|        4|       104.0|        95|  2375|        17.5|        70|     2|            saab 99e|
| 9.0|        8|       304.0|       193|  4732|        18.5|        70|     1|            hi 1200d|


In [24]:
df.filter(df['mpg']>23).show(10)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|24.0|        4|       113.0|        95|  2372|        15.0|        70|     3|toyota corona mar...|
|27.0|        4|        97.0|        88|  2130|        14.5|        70|     3|        datsun pl510|
|26.0|        4|        97.0|        46|  1835|        20.5|        70|     2|volkswagen 1131 d...|
|25.0|        4|       110.0|        87|  2672|        17.5|        70|     2|         peugeot 504|
|24.0|        4|       107.0|        90|  2430|        14.5|        70|     2|         audi 100 ls|
|25.0|        4|       104.0|        95|  2375|        17.5|        70|     2|            saab 99e|
|26.0|        4|       121.0|       113|  2234|        12.5|        70|     2|            bmw 2002|


In [25]:
df.filter((col('weight')>2000) & (col('horsepower')>80)).show(10)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|         ford torino|
|15.0|        8|       429.0|       198|  4341|        10.0|        70|     1|    ford galaxie 500|
|14.0|        8|       454.0|       220|  4354|         9.0|        70|     1|    chevrolet impala|


In [26]:
df.filter((col('weight')>2000) & (col('horsepower')>80)).sort('mpg').show(10)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
| 9.0|        8|       304.0|       193|  4732|        18.5|        70|     1|            hi 1200d|
|10.0|        8|       360.0|       215|  4615|        14.0|        70|     1|           ford f250|
|10.0|        8|       307.0|       200|  4376|        15.0|        70|     1|           chevy c20|
|11.0|        8|       318.0|       210|  4382|        13.5|        70|     1|          dodge d200|
|11.0|        8|       429.0|       208|  4633|        11.0|        72|     1|     mercury marquis|
|11.0|        8|       400.0|       150|  4997|        14.0|        73|     1|    chevrolet impala|
|11.0|        8|       350.0|       180|  3664|        11.0|        73|     1|    oldsmobile omega|


In [27]:
df.filter((col('weight')>2000) & (col('horsepower')>80)).sort('mpg').show(10)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
| 9.0|        8|       304.0|       193|  4732|        18.5|        70|     1|            hi 1200d|
|10.0|        8|       360.0|       215|  4615|        14.0|        70|     1|           ford f250|
|10.0|        8|       307.0|       200|  4376|        15.0|        70|     1|           chevy c20|
|11.0|        8|       318.0|       210|  4382|        13.5|        70|     1|          dodge d200|
|11.0|        8|       429.0|       208|  4633|        11.0|        72|     1|     mercury marquis|
|11.0|        8|       400.0|       150|  4997|        14.0|        73|     1|    chevrolet impala|
|11.0|        8|       350.0|       180|  3664|        11.0|        73|     1|    oldsmobile omega|


In [28]:
df.filter((col('weight')>2000) & (col('horsepower')>80)).sort('mpg').show(10)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
| 9.0|        8|       304.0|       193|  4732|        18.5|        70|     1|            hi 1200d|
|10.0|        8|       360.0|       215|  4615|        14.0|        70|     1|           ford f250|
|10.0|        8|       307.0|       200|  4376|        15.0|        70|     1|           chevy c20|
|11.0|        8|       318.0|       210|  4382|        13.5|        70|     1|          dodge d200|
|11.0|        8|       429.0|       208|  4633|        11.0|        72|     1|     mercury marquis|
|11.0|        8|       400.0|       150|  4997|        14.0|        73|     1|    chevrolet impala|
|11.0|        8|       350.0|       180|  3664|        11.0|        73|     1|    oldsmobile omega|


In [29]:
df.filter((col('weight')>2000) & (col('horsepower')>80)).sort(col('mpg').desc()).show(10)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|38.0|        6|       262.0|        85|  3015|        17.0|        82|     1|oldsmobile cutlas...|
|37.0|        4|       119.0|        92|  2434|        15.0|        80|     3|datsun 510 hatchback|
|36.0|        4|       135.0|        84|  2370|        13.0|        82|     1|   dodge charger 2.2|
|36.0|        4|       120.0|        88|  2160|        14.5|        82|     3|    nissan stanza xe|
|35.0|        4|       122.0|        88|  2500|        15.1|        80|     2|   triumph tr7 coupe|
|34.0|        4|       112.0|        88|  2395|        18.0|        82|     1|chevrolet cavalie...|
|33.5|        4|        98.0|        83|  2075|        15.9|        77|     1|      dodge colt m/m|


In [30]:
df.sort('model year').show(10)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|         ford torino|
|15.0|        8|       429.0|       198|  4341|        10.0|        70|     1|    ford galaxie 500|
|14.0|        8|       454.0|       220|  4354|         9.0|        70|     1|    chevrolet impala|


In [31]:
df.filter(col("car name").contains("volkswagen")).orderBy(['model year','horsepower'],ascending=[False,False]).show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|36.0|        4|       105.0|        74|  1980|        15.3|        82|     2| volkswagen rabbit l|
|33.0|        4|       105.0|        74|  2190|        14.2|        81|     2|    volkswagen jetta|
|31.5|        4|        89.0|        71|  1990|        14.9|        78|     2| volkswagen scirocco|
|43.1|        4|        90.0|        48|  1985|        21.5|        78|     2|volkswagen rabbit...|
|29.0|        4|        97.0|        78|  1940|        14.5|        77|     2|volkswagen rabbit...|
|30.5|        4|        97.0|        78|  2190|        14.1|        77|     2|   volkswagen dasher|
|29.5|        4|        97.0|        71|  1825|        12.2|        76|     2|   volkswagen rabbit|


In [32]:
df =df.withColumn('brand',split(col('car name'),' ').getItem(0))

auto_misspelled = {'chevroelt':'chevrolet',
                   'chevy':'chevrolet',
                   'vw':'volkswagan',
                   'volkswagen':'volkswagan',
                   'vokswagen':'volkswagan',
                   'toyouta':'toyota',
                   'hi':'harvester',
                   'maxda':'mazda',
                   'mercedes-benz':'mercedes'}

for key in auto_misspelled.keys():
  df = df.withColumn('brand', regexp_replace('brand', key, auto_misspelled[key]))
df.show(20)

+----+---------+------------+----------+------+------------+----------+------+--------------------+----------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|     brand|
+----+---------+------------+----------+------+------------+----------+------+--------------------+----------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...| chevrolet|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|     buick|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|  plymouth satellite|  plymouth|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|       amc rebel sst|       amc|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|         ford torino|      ford|
|15.0|        8|       429.0|       198|  4341|        10.0|        70|     1|    ford galaxie 500|      ford|
|

In [33]:

df.groupBy('brand').agg({'acceleration':'mean'}).withColumnRenamed('avg(acceleration)','Avg acceleration').show(10)


+----------+------------------+
|     brand|  Avg acceleration|
+----------+------------------+
|volkswagan|17.404545454545453|
|     buick|14.700000000000003|
|   pontiac|14.081249999999999|
|  mercedes| 19.53333333333333|
|    toyota| 16.03846153846154|
|      saab|            15.175|
|       amc|15.264285714285716|
|   peugeot|             19.55|
|  chrysler|13.300000000000002|
|  plymouth|14.851612903225801|
+----------+------------------+
only showing top 10 rows



In [34]:
df.groupBy('brand').agg({'mpg':'max'}).show(10)

+----------+--------+
|     brand|max(mpg)|
+----------+--------+
|volkswagan|    44.3|
|     buick|    30.0|
|   pontiac|    33.5|
|  mercedes|    30.0|
|    toyota|    39.1|
|      saab|    25.0|
|       amc|    27.4|
|   peugeot|    30.0|
|  chrysler|    26.0|
|  plymouth|    39.0|
+----------+--------+
only showing top 10 rows



In [35]:
df.groupBy('brand').agg({'acceleration':'mean'}).show(10)

+----------+------------------+
|     brand| avg(acceleration)|
+----------+------------------+
|volkswagan|17.404545454545453|
|     buick|14.700000000000003|
|   pontiac|14.081249999999999|
|  mercedes| 19.53333333333333|
|    toyota| 16.03846153846154|
|      saab|            15.175|
|       amc|15.264285714285716|
|   peugeot|             19.55|
|  chrysler|13.300000000000002|
|  plymouth|14.851612903225801|
+----------+------------------+
only showing top 10 rows



In [36]:
distinct_brands = df.select('brand').distinct().count()
print(f"Number of distinct brands: {distinct_brands}")

df.select('brand').distinct().show(100,truncate=False)

Number of distinct brands: 30
+----------+
|brand     |
+----------+
|volkswagan|
|buick     |
|pontiac   |
|mercedes  |
|toyota    |
|saab      |
|amc       |
|peugeot   |
|chrysler  |
|plymouth  |
|audi      |
|datsun    |
|bmw       |
|dodge     |
|ford      |
|capri     |
|cadillac  |
|mazda     |
|harvester |
|chevrolet |
|opel      |
|honda     |
|fiat      |
|oldsmobile|
|mercury   |
|renault   |
|nissan    |
|triumph   |
|subaru    |
|volvo     |
+----------+



In [37]:
# prompt: command to generate unique values in brand

#from pyspark.sql.functions import countDistinct

# Assuming 'df' is your DataFrame as defined in the previous code

#distinct_brands = df.select(countDistinct("brand")).collect()[0][0]
#print(f"Number of distinct brands: {distinct_brands}")
#df.select("brand").distinct().show(distinct_brands)
#df.select('brand').distinct().show()


def one_hot_encode(df, categorical_col):
  # Create a StringIndexer
  indexed = StringIndexer().setInputCol(categorical_col).setOutputCol(categorical_col + "_cat").fit(df).transform(df)
  # Create a OneHotEncoder
  encoder = OneHotEncoder().setInputCol(categorical_col + "_cat").setOutputCol(categorical_col + "_OneHotEncoder").fit(indexed).transform(indexed)

  ohe = encoder.drop(*[categorical_col, categorical_col + '_cat'])

  return ohe

df = one_hot_encode(df, 'brand')




In [38]:
df.show(50)

+----+---------+------------+----------+------+------------+----------+------+--------------------+-------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|brand_OneHotEncoder|
+----+---------+------------+----------+------+------------+----------+------+--------------------+-------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...|     (29,[1],[1.0])|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|     (29,[8],[1.0])|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|  plymouth satellite|     (29,[2],[1.0])|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|       amc rebel sst|     (29,[3],[1.0])|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|         ford torino|     (29,[0],[1.0])|
|15.0|        8|       429.0|       198|

In [39]:
df=df.drop('car name')
df.show(10)

+----+---------+------------+----------+------+------------+----------+------+-------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|brand_OneHotEncoder|
+----+---------+------------+----------+------+------------+----------+------+-------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|     (29,[1],[1.0])|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|     (29,[8],[1.0])|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|     (29,[2],[1.0])|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|     (29,[3],[1.0])|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|     (29,[0],[1.0])|
|15.0|        8|       429.0|       198|  4341|        10.0|        70|     1|     (29,[0],[1.0])|
|14.0|        8|       454.0|       220|  4354|         9.0|        70|     1|     (29,[1],[1.0])|
|14.0|    

In [40]:
from pyspark.ml.feature import VectorAssembler

def vector_assembler(df,indep_cols):
  assembler = VectorAssembler(inputCols=indep_cols,
                              outputCol = 'features',
                              handleInvalid="skip")
  output = assembler.transform(df).drop(*indep_cols)
  return output

df = vector_assembler(df,indep_cols=df.drop('mpg').columns)


In [41]:
df.show(100,truncate=False)

+----+------------------------------------------------------------------+
|mpg |features                                                          |
+----+------------------------------------------------------------------+
|18.0|(36,[0,1,2,3,4,5,6,8],[8.0,307.0,130.0,3504.0,12.0,70.0,1.0,1.0]) |
|15.0|(36,[0,1,2,3,4,5,6,15],[8.0,350.0,165.0,3693.0,11.5,70.0,1.0,1.0])|
|18.0|(36,[0,1,2,3,4,5,6,9],[8.0,318.0,150.0,3436.0,11.0,70.0,1.0,1.0]) |
|16.0|(36,[0,1,2,3,4,5,6,10],[8.0,304.0,150.0,3433.0,12.0,70.0,1.0,1.0])|
|17.0|(36,[0,1,2,3,4,5,6,7],[8.0,302.0,140.0,3449.0,10.5,70.0,1.0,1.0]) |
|15.0|(36,[0,1,2,3,4,5,6,7],[8.0,429.0,198.0,4341.0,10.0,70.0,1.0,1.0]) |
|14.0|(36,[0,1,2,3,4,5,6,8],[8.0,454.0,220.0,4354.0,9.0,70.0,1.0,1.0])  |
|14.0|(36,[0,1,2,3,4,5,6,9],[8.0,440.0,215.0,4312.0,8.5,70.0,1.0,1.0])  |
|14.0|(36,[0,1,2,3,4,5,6,16],[8.0,455.0,225.0,4425.0,10.0,70.0,1.0,1.0])|
|15.0|(36,[0,1,2,3,4,5,6,10],[8.0,390.0,190.0,3850.0,8.5,70.0,1.0,1.0]) |
|15.0|(36,[0,1,2,3,4,5,6,11],[8.0,383.

In [42]:
train_data,test_data = df.randomSplit([0.8,0.2],seed=123)

print(f"Train data shape: {train_data.count()}, {len(train_data.columns)}")
print(f"Test data shape: {test_data.count()}, {len(test_data.columns)}")

Train data shape: 318, 2
Test data shape: 80, 2


In [43]:
lr = LinearRegression(featuresCol='features',labelCol='mpg',regParam=0.3,standardization=False)
lr_model = lr.fit(train_data)

In [44]:
def eval_model(model,test_data):
  print(model.__class__.__name__.center(70,'='))
  model_result = model.evaluate(test_data)
  print(f"RMSE: {model_result.rootMeanSquaredError}")
  print(f"R2: {model_result.r2}")
  print(f"MAE: {model_result.meanAbsoluteError}")
  print(f"MSE: {model_result.meanSquaredError}")
  print(f"Explained Variance: {model_result.explainedVariance}")
  print(70*'=')

eval_model(lr_model,test_data)


========================LinearRegressionModel=========================
RMSE: 3.412223445197354
R2: 0.7983309966898102
MAE: 2.461164491835024
MSE: 11.643268839954501
Explained Variance: 43.63256309984281


In [45]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print(sc)

<SparkContext master=local[*] appName=First Session>


##RDDs

In [46]:
rdd = sc.emptyRDD()
rdd.isEmpty()
#print(rdd)

True

In [47]:
a = sc.parallelize([])
a.isEmpty()

True

In [48]:
mixRDD = sc.parallelize([True,[11,22,33,44,55],(10,20,30,40,50)],3)
mixRDD.collect()

[True, [11, 22, 33, 44, 55], (10, 20, 30, 40, 50)]

In [49]:
mixRDD.getNumPartitions()

3

In [50]:
mixRDD.glom().collect()

[[True], [[11, 22, 33, 44, 55]], [(10, 20, 30, 40, 50)]]

In [51]:
mixRDD.setName('MyRDD')

MyRDD ParallelCollectionRDD[189] at readRDDFromFile at PythonRDD.scala:289

In [52]:
import random
random.randint(1, 1000)

524

In [53]:
#a = sc.parallelize([random.randint(1, 6) for _ in range(20)],4)
a = sc.parallelize(range(1,1000),4)
print(a.getNumPartitions())
print("\n")
print(a.glom().take(4))
print("\n")
print(a.glom().min()) # to get min partitions data means first one partition(0)
print("\n")
print(a.glom().max()) # to get max partitions data means last one partition(3)

4


[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 22

In [54]:
b = sc.parallelize([12,21,23,43,1,22,11,45,56])
print(b.takeOrdered(4))
print(b.takeOrdered(4,key = lambda x:-x))

[1, 11, 12, 21]
[56, 45, 43, 23]


In [55]:
a.glom().map(lambda x:max(x)).collect()

[249, 499, 749, 999]

In [56]:
for i in range(a.getNumPartitions()):
  print(f"Max of partition({i}) : {max(a.glom().collect()[i])}")
  print(f"Min of partition({i}) : {min(a.glom().collect()[i])}")

Max of partition(0) : 249
Min of partition(0) : 1
Max of partition(1) : 499
Min of partition(1) : 250
Max of partition(2) : 749
Min of partition(2) : 500
Max of partition(3) : 999
Min of partition(3) : 750


##Repartitioning and Coalesce

In [57]:
d= b.repartition(7)
d.getNumPartitions()

7

In [58]:
e = d.repartition(4)
e.getNumPartitions()

4

In [59]:
c=b
e = d.coalesce(2)
e.getNumPartitions()

2

In [88]:
file_path='/content/UberDatasetCleaned.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True,nanValue='?')

In [62]:
df.show()

+----------+----------+------------+----------+--------+----------+-----+----+--------+-----------------+---------------+----------+---------------+--------+----------------+---------+---------+
|start_date|start_hour|start_minute|  end_date|end_hour|end_minute|month|year|CATEGORY|            START|           STOP|     MILES|        PURPOSE|duration|           speed|      day| day_time|
+----------+----------+------------+----------+--------+----------+-----+----+--------+-----------------+---------------+----------+---------------+--------+----------------+---------+---------+
|2016-01-01|        21|          11|2016-01-01|      21|        17|  Jan|2016|Business|      Fort Pierce|    Fort Pierce|  8.207634| Meal/Entertain|       6|        82.07634|   Friday|  Evening|
|2016-01-02|         1|          25|2016-01-02|       1|        37|  Jan|2016|Business|      Fort Pierce|    Fort Pierce|    8.0467| Meal/Entertain|      12|         40.2335| Saturday|    Night|
|2016-01-02|        20|  

In [63]:
def check_missing(dataframe):
  """
  Checks for missing values in each column of a DataFrame and displays the count.

  Args:
    dataframe: The input DataFrame.
  """
  # Iterate through columns and handle different data types
  for c in dataframe.columns:
    if dataframe.schema[c].dataType in [IntegerType(), FloatType()]:
      # Use isnan and isNull for numeric types
      missing_count = dataframe.select(count(when(isnan(col(c)) | col(c).isNull(), c))).first()[0]
      print(f"Missing values in column '{c}': {missing_count}")
    else:
      # Use isNull for non-numeric types
      missing_count = dataframe.select(count(when(col(c).isNull(), c))).first()[0]
      print(f"Missing values in column '{c}': {missing_count}")

check_missing(df)

Missing values in column 'start_date': 0
Missing values in column 'start_hour': 0
Missing values in column 'start_minute': 0
Missing values in column 'end_date': 0
Missing values in column 'end_hour': 0
Missing values in column 'end_minute': 0
Missing values in column 'month': 0
Missing values in column 'year': 0
Missing values in column 'CATEGORY': 0
Missing values in column 'START': 0
Missing values in column 'STOP': 0
Missing values in column 'MILES': 0
Missing values in column 'PURPOSE': 0
Missing values in column 'duration': 0
Missing values in column 'speed': 0
Missing values in column 'day': 0
Missing values in column 'day_time': 0


Q1. Identify most frequent travel routes\
Q2. Analyze peak travel hours\
Q3. find the longest business trips\
Q4. Find the avg speed bassed on daytime\
Q5. Identify most common travel apttern by day of te week\
Q6. analyze travel pattern by day of the week\
Q7. Find the best customer(who travel  more frequently than others)\
Q8. identify each customer most frequent start location\
Q9. Identify shortest business route\
Q10.

##Q1 Identify most frequent travel routes

In [89]:
from pyspark.sql.functions import concat, lit

df = df.withColumn('route', concat(df['START'], lit('-'), df['STOP']))

In [65]:
#df = df.withColumn('route', pyspark.sql.functions.concat(df['START'],df['STOP']))
df.show()

+----------+----------+------------+----------+--------+----------+-----+----+--------+-----------------+---------------+----------+---------------+--------+----------------+---------+---------+--------------------+
|start_date|start_hour|start_minute|  end_date|end_hour|end_minute|month|year|CATEGORY|            START|           STOP|     MILES|        PURPOSE|duration|           speed|      day| day_time|               route|
+----------+----------+------------+----------+--------+----------+-----+----+--------+-----------------+---------------+----------+---------------+--------+----------------+---------+---------+--------------------+
|2016-01-01|        21|          11|2016-01-01|      21|        17|  Jan|2016|Business|      Fort Pierce|    Fort Pierce|  8.207634| Meal/Entertain|       6|        82.07634|   Friday|  Evening|Fort Pierce-Fort ...|
|2016-01-02|         1|          25|2016-01-02|       1|        37|  Jan|2016|Business|      Fort Pierce|    Fort Pierce|    8.0467| Mea

In [66]:
#filtered_df = df.where(df['START'] != 'Unknown Location')
#filtered_df.show()

+----------+----------+------------+----------+--------+----------+-----+----+--------+-----------------+---------------+----------+---------------+--------+----------------+---------+---------+--------------------+
|start_date|start_hour|start_minute|  end_date|end_hour|end_minute|month|year|CATEGORY|            START|           STOP|     MILES|        PURPOSE|duration|           speed|      day| day_time|               route|
+----------+----------+------------+----------+--------+----------+-----+----+--------+-----------------+---------------+----------+---------------+--------+----------------+---------+---------+--------------------+
|2016-01-01|        21|          11|2016-01-01|      21|        17|  Jan|2016|Business|      Fort Pierce|    Fort Pierce|  8.207634| Meal/Entertain|       6|        82.07634|   Friday|  Evening|Fort Pierce-Fort ...|
|2016-01-02|         1|          25|2016-01-02|       1|        37|  Jan|2016|Business|      Fort Pierce|    Fort Pierce|    8.0467| Mea

In [67]:
#df= filtered_df

In [90]:
route_counts = df.where(df['START'] != 'Unknown Location').groupBy('route').count()
route_counts.show(2000)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|Meredith Townes-H...|    1|
|San Francisco-Pal...|    1|
| Berkeley-Menlo Park|    1|
|R?walpindi-Unknow...|    8|
|Edgehill Farms-Wh...|    6|
|  Katunayake-Gampaha|    1|
|         Cary-Durham|   36|
|Red River Distric...|    1|
|South Congress-No...|    1|
|   Coxville-The Drag|    1|
|Cary-Unknown Loca...|    1|
|   Orlando-Kissimmee|    7|
|Tribeca-Financial...|    1|
|Financial Distric...|    1|
|Heritage Pines-Wh...|    1|
|San Francisco-Ber...|    4|
|Lower Manhattan-H...|    1|
|Whitebridge-Savon...|    3|
|Central-College A...|    1|
|Berkeley-Mountain...|    1|
|Midtown-Midtown West|    1|
| Whitebridge-Preston|    3|
|Seaport-Gramercy-...|    1|
|   Berkeley-San Jose|    1|
|    Berkeley-Oakland|    3|
|         CBD-Bywater|    1|
|San Francisco-Oak...|    2|
| SOMISSPO-Tenderloin|    1|
|Kar?chi-Unknown L...|    7|
|Hudson Square-Low...|    1|
|Fairmont-Meredith...|    1|
|   Galveston-

In [91]:
route_counts.orderBy('count',ascending=False).show(10)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|    Morrisville-Cary|   74|
|    Cary-Morrisville|   67|
|           Cary-Cary|   53|
|         Cary-Durham|   36|
|Islamabad-Unknown...|   28|
|       Lahore-Lahore|   27|
| Islamabad-Islamabad|   25|
|         Durham-Cary|   25|
|        Cary-Raleigh|   23|
|        Raleigh-Cary|   21|
+--------------------+-----+
only showing top 10 rows



##Q2 Analyze peak travel hours

In [72]:
#df = df.withColumn('peak_travel', concat(df['start_hour'], lit('-'), df['end_hour']))

In [92]:
df.groupBy('start_hour').count().orderBy('count',ascending=False).show(10)

+----------+-----+
|start_hour|count|
+----------+-----+
|        15|   98|
|        17|   95|
|        18|   94|
|        13|   94|
|        14|   89|
|        16|   88|
|        12|   77|
|        11|   72|
|        20|   71|
|        19|   68|
+----------+-----+
only showing top 10 rows



##Q3 find the longest business trips

In [101]:
df.where(df.CATEGORY== 'Business').orderBy('duration',ascending=False).show(1)

+----------+----------+------------+----------+--------+----------+-----+----+--------+-----+------------+----------+--------------+--------+----------------+------+---------+------------------+
|start_date|start_hour|start_minute|  end_date|end_hour|end_minute|month|year|CATEGORY|START|        STOP|     MILES|       PURPOSE|duration|           speed|   day| day_time|             route|
+----------+----------+------------+----------+--------+----------+-----+----+--------+-----+------------+----------+--------------+--------+----------------+------+---------+------------------+
|2016-03-25|        16|          52|2016-03-25|      22|        22|  Mar|2016|Business|Latta|Jacksonville|499.378202|Customer Visit|     330|90.7960367272727|Friday|Afternoon|Latta-Jacksonville|
+----------+----------+------------+----------+--------+----------+-----+----+--------+-----+------------+----------+--------------+--------+----------------+------+---------+------------------+
only showing top 1 row



##Q4 Find the avg speed bassed on daytime

DataFrame[day_time: string]

In [95]:
df = df.where(df.duration > 0)

In [96]:
from pyspark.sql.functions import avg

df.groupBy('day_time').agg(avg('speed').alias('avg_speed')).show()

+---------+------------------+
| day_time|         avg_speed|
+---------+------------------+
|  Evening|  43.4256712248582|
|  Morning| 47.72446648096086|
|Afternoon| 39.87152502771442|
|    Night|46.036096350678385|
+---------+------------------+



##Q5 Identify most common trip purpose

In [100]:
df.groupBy('PURPOSE').count().orderBy('count',ascending=False).show(1)

+-------+-----+
|PURPOSE|count|
+-------+-----+
|Meeting|  295|
+-------+-----+
only showing top 1 row



##Q6. Analyze travel pattern by the day of the week

In [102]:
df.groupBy('day').count().orderBy('count',ascending=False).show()

+---------+-----+
|      day|count|
+---------+-----+
|   Friday|  201|
|   Monday|  174|
|  Tuesday|  172|
| Thursday|  152|
|   Sunday|  147|
|Wednesday|  146|
| Saturday|  145|
+---------+-----+



##Q7. Identify most frequent start location with respect to daytime

Simplest way

In [127]:
ans = df.where(df.START != 'Unknown Location').groupBy('START','day_time').count().orderBy('count',ascending=False)

In [126]:
ans.where(ans.day_time == 'Morning').orderBy('count',ascending=False).show(1)
ans.where(ans.day_time == 'Night').orderBy('count',ascending=False).show(1)
ans.where(ans.day_time == 'Afternoon').orderBy('count',ascending=False).show(1)
ans.where(ans.day_time == 'Evening').orderBy('count',ascending=False).show(1)

+-----+--------+-----+
|START|day_time|count|
+-----+--------+-----+
| Cary| Morning|   62|
+-----+--------+-----+
only showing top 1 row

+-----------+--------+-----+
|      START|day_time|count|
+-----------+--------+-----+
|Morrisville|   Night|    8|
+-----------+--------+-----+
only showing top 1 row

+-----+---------+-----+
|START| day_time|count|
+-----+---------+-----+
| Cary|Afternoon|   96|
+-----+---------+-----+
only showing top 1 row

+-----+--------+-----+
|START|day_time|count|
+-----+--------+-----+
| Cary| Evening|   38|
+-----+--------+-----+
only showing top 1 row



One way

In [142]:
from pyspark.sql.functions import max
filtered_df= df.where(df['START']!= 'Unknown Location')
start_location= filtered_df.groupBy('day_time','START').count().orderBy('count', ascending=False)

from pyspark.sql import functions as F
from pyspark.sql.window import Window
window_spec = Window.partitionBy('day_time').orderBy(F.col('count').desc())
ranked_df = start_location.withColumn('rank', F.row_number().over(window_spec))
most_frequent_locations = ranked_df.filter(ranked_df['rank'] == 1).drop('rank')

most_frequent_locations.show()

+---------+-----------+-----+
| day_time|      START|count|
+---------+-----------+-----+
|Afternoon|       Cary|   96|
|  Evening|       Cary|   38|
|  Morning|       Cary|   62|
|    Night|Morrisville|    8|
+---------+-----------+-----+



Another way

In [175]:
d2 = df.where(df.START != 'Unknown Location').groupBy('START','day_time').count().orderBy('count',ascending=False)
Max_counts = d2.groupBy('day_time').agg(max('count').alias('Max_count'))

d2_alias = d2.alias('d2')
Max_counts_alias = Max_counts.alias('Max_counts')
result = d2.join(Max_counts,(d2['day_time'] == Max_counts['day_time']) & (d2['count'] == Max_counts['Max_count'])).select(d2_alias['day_time'], d2_alias['START'], d2_alias['count'])
result.show()

+---------+-----------+-----+
| day_time|      START|count|
+---------+-----------+-----+
|  Evening|       Cary|   38|
|  Morning|       Cary|   62|
|Afternoon|       Cary|   96|
|    Night|Morrisville|    8|
+---------+-----------+-----+



In [167]:
d2.show()
d2_alias.show()

+--------------+---------+-----+
|         START| day_time|count|
+--------------+---------+-----+
|          Cary|Afternoon|   96|
|          Cary|  Morning|   62|
|   Whitebridge|Afternoon|   39|
|          Cary|  Evening|   38|
|   Morrisville|  Evening|   36|
|     Islamabad|Afternoon|   34|
|   Morrisville|Afternoon|   30|
|       Raleigh|Afternoon|   19|
|   Whitebridge|  Morning|   17|
|        Durham|Afternoon|   17|
|     Islamabad|  Morning|   16|
|Westpark Place|Afternoon|   13|
|        Lahore|Afternoon|   13|
|       Kar?chi|Afternoon|   12|
|        Lahore|  Morning|   12|
|   Whitebridge|  Evening|   11|
|          Apex|Afternoon|   11|
|   Morrisville|  Morning|   10|
|        Lahore|  Evening|   10|
|       Kar?chi|  Morning|    8|
+--------------+---------+-----+
only showing top 20 rows

+--------------+---------+-----+
|         START| day_time|count|
+--------------+---------+-----+
|          Cary|Afternoon|   96|
|          Cary|  Morning|   62|
|   Whitebridge|A

##Q8. Identify shortest business route

In [178]:
d = df.where(df.CATEGORY == 'Business').orderBy('MILES',ascending=True).first()
df.where(df.MILES == d.MILES).show()

+----------+----------+------------+----------+--------+----------+-----+----+--------+----------+----------+-------+---------------+--------+-------+---------+---------+--------------------+
|start_date|start_hour|start_minute|  end_date|end_hour|end_minute|month|year|CATEGORY|     START|      STOP|  MILES|        PURPOSE|duration|  speed|      day| day_time|               route|
+----------+----------+------------+----------+--------+----------+-----+----+--------+----------+----------+-------+---------------+--------+-------+---------+---------+--------------------+
|2016-01-26|        17|          27|2016-01-26|      17|        29|  Jan|2016|Business|      Cary|      Cary|0.80467|Errand/Supplies|       2|24.1401|  Tuesday|Afternoon|           Cary-Cary|
|2016-02-17|        16|          38|2016-02-17|      16|        43|  Feb|2016|Business|Katunayaka|Katunayaka|0.80467|Errand/Supplies|       5|9.65604|Wednesday|Afternoon|Katunayaka-Katuna...|
|2016-06-08|        17|          16|2016

In [184]:
d.MILES

0.80467

In [183]:
df.select(df[0]).show()

+----------+
|start_date|
+----------+
|2016-01-01|
|2016-01-02|
|2016-01-02|
|2016-01-05|
|2016-01-06|
|2016-01-06|
|2016-01-06|
|2016-01-07|
|2016-01-10|
|2016-01-10|
|2016-01-10|
|2016-01-10|
|2016-01-10|
|2016-01-11|
|2016-01-11|
|2016-01-11|
|2016-01-11|
|2016-01-12|
|2016-01-12|
|2016-01-12|
+----------+
only showing top 20 rows

